In [24]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from difflib import SequenceMatcher
import Levenshtein

# Read recipe inputs
CALCULATED_DRAW_DOWNS = dataiku.Dataset("CALCULATED_DRAW_DOWNS")
CALCULATED_DRAW_DOWNS_df = CALCULATED_DRAW_DOWNS.get_dataframe()

CALCULATED_DRAW_UPS = dataiku.Dataset("CALCULATED_DRAW_UPS")
CALCULATED_DRAW_UPS_df = CALCULATED_DRAW_UPS.get_dataframe()

COMMON_WORDS = dataiku.Dataset("NAFCUSTOMER_COMMON_WORDS_IN_NAMES")
COMMON_WORDS_df = COMMON_WORDS.get_dataframe()

In [25]:
def date_tz_naive(pd_s):
    return pd.to_datetime(pd_s).apply(lambda x:x.tz_localize(None))

In [29]:
# consider customers with the card threshold or more
# set this too low and the running time will balloon
# recommend: 10 or higher

card_cut_off_threshold = 5

df_down_full = CALCULATED_DRAW_DOWNS_df[CALCULATED_DRAW_DOWNS_df.ACTIVE_CARD_MAX>card_cut_off_threshold].copy()
df_up_full = CALCULATED_DRAW_UPS_df[CALCULATED_DRAW_UPS_df.ACTIVE_CARD_MAX>card_cut_off_threshold].copy()

df_down_full.DRAW_DOWN_DATE = date_tz_naive(df_down_full['DRAW_DOWN_DATE'])
df_up_full.DRAW_UP_DATE = date_tz_naive(df_up_full['DRAW_UP_DATE'])

df_up_full.dropna(subset=['DRAW_UP_DATE'], inplace=True)

df_common = COMMON_WORDS_df

df_down_full.sort_values(['CUSTOMER'], inplace=True)
df_up_full.sort_values(['CUSTOMER'], inplace=True)
df_common.sort_values(['WORD'], inplace=True)

print(len(df_down_full), "draw downs full")
print(len(df_up_full), "draw ups full")
print(len(df_common), "common words")

34027 draw downs full
72496 draw ups full
2173 common words


In [30]:
import string

_common_words = df_common.WORD.unique()
print(len(_common_words), "screening against common words")

class Draw_Down_Customer:

    def __init__(self, name, draw_down_date, active_card_max):

        self.CUSTOMER = name
        self.DRAW_DOWN_DATE = draw_down_date
        self.ACTIVE_CARD_MAX = active_card_max

        self.MATCHING_CUSTOMERS = []
        self.PERCENT_DIFFERENCE = []
        self.DAYS_DIFFERENCE = []
        self.DRAW_UP_DATE = []

        # remove punctuation
        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()
        self.WORD_LIST = []
        for w in f:
            if w not in _common_words:
                self.WORD_LIST.append(w)

    def Match_Draw_Up_Customer(self, name, draw_up_date, active_card_max):

        if (self.CUSTOMER == name):
            # exact match, already captured
            return

        c_str = name.translate(str.maketrans('', '', string.punctuation))

        f = c_str.split()

        check_list = []
        for w in f:
            if (w not in _common_words) and (len(w)>1) and (not w.isnumeric()):
                check_list.append(w)

        percent_diff = round((abs(self.ACTIVE_CARD_MAX - active_card_max) / ((self.ACTIVE_CARD_MAX+active_card_max)/2)),2)

        #date_format = "%Y-%m-%d"
        #d1_date = datetime.strptime(draw_up_date.astype(str), date_format)
        #d2_date = datetime.strptime(self.DRAW_DOWN_DATE.astype(str), date_format)

        delta_between_drop_and_rise = round(abs((draw_up_date-self.DRAW_DOWN_DATE).days)/30.,0)

        for w_to_check in check_list:
            for w in self.WORD_LIST:
                if w_to_check==w:

                    if not name in(self.MATCHING_CUSTOMERS) and(delta_between_drop_and_rise<=4)and(percent_diff<=0.5) :
                        self.MATCHING_CUSTOMERS.append(name)
                        self.PERCENT_DIFFERENCE.append(percent_diff)
                        self.DAYS_DIFFERENCE.append(delta_between_drop_and_rise)
                        self.DRAW_UP_DATE.append(draw_up_date)
                        break;

2173 screening against common words


In [31]:
import time

def do_save_log(_matching_process_log_time, _matching_process_log_event):

    df_matching_log = pd.DataFrame(_matching_process_log_time)
    if len(df_matching_log)>0:

        df_matching_log.columns = ['LOG_TIME']
        df_matching_log['LOG_EVENT'] = _matching_process_log_event

        MATCHING_PROCESS_LOG_V_df = df_matching_log
        MATCHING_PROCESS_LOG_V = dataiku.Dataset("MATCHING_PROCESS_LOG_V")
        MATCHING_PROCESS_LOG_V.write_with_schema(MATCHING_PROCESS_LOG_V_df)

        print()

def do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date):

    df_matches = pd.DataFrame(_direct_customer)
    if len(df_matches)>0:

        print()
        print("saving", len(df_matches), "1-1 matching records")
        print()

        df_matches.columns = ['CUSTOMER']
        df_matches["MATCH_CUSTOMER"] = _direct_match
        df_matches["DRAW_UP_DATE"] = _direct_draw_up_date

        MATCHES_1_TO_1_STAGING_V_df = df_matches
        MATCHES_1_TO_1_STAGING_V = dataiku.Dataset("MATCHES_1_TO_1_STAGING_V")
        MATCHES_1_TO_1_STAGING_V.write_with_schema(MATCHES_1_TO_1_STAGING_V_df)

        print()

def do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates):

    df_multiple_matches = pd.DataFrame(_multiple_customer)

    if len(df_multiple_matches)>0:

        print()
        print("saving", len(df_multiple_matches), "1-n matching records")
        print()

        df_multiple_matches.columns = ['CUSTOMER']
        df_multiple_matches["MATCH_CUSTOMER"] = _multiple_matches
        df_multiple_matches["DRAW_UP_DATE"] = _multiple_drop_dates

        MATCHES_1_TO_N_STAGING_V_df = df_multiple_matches
        MATCHES_1_TO_N_STAGING_V = dataiku.Dataset("MATCHES_1_TO_N_STAGING_V")
        MATCHES_1_TO_N_STAGING_V.write_with_schema(MATCHES_1_TO_N_STAGING_V_df)

        print()

In [32]:
from datetime import timedelta

df_down = df_down_full
df_up = df_up_full

_processed_customers = []
verbose = False

_matching_process_log_time = []
_matching_process_log_event = []

_direct_customer = []
_direct_match = []
_direct_draw_up_date = []

_multiple_customer = []
_multiple_matches = []
_multiple_drop_dates = []

_no_match_customer = []

save_every_n = 50
to_save_counter = 0
print_every_n = 100

print(len(df_down), "filtered down rows")
print(len(df_up), "filtered up rows")

_customers = []

t0 = time.time()

for index, row in df_down.iterrows():

    customer = row['CUSTOMER']
    draw_down_date = row['DRAW_DOWN_DATE']
    active_card_max = row['ACTIVE_CARD_MAX']

    c = Draw_Down_Customer(customer, draw_down_date, active_card_max)

    _customers.append(c)

idx = 0

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append(" processing range " + str(len(_customers)) + " Draw Down Customers")
do_save_log(_matching_process_log_time, _matching_process_log_event)


34027 filtered down rows
72496 filtered up rows
1 rows successfully written (YNkvUBkUxF)



In [0]:
idx=0
for c in _customers:

    idx+=1
    
    date_start = pd.to_datetime(c.DRAW_DOWN_DATE) +timedelta(days=-120)
    date_end = pd.to_datetime(c.DRAW_DOWN_DATE) +timedelta(days=120)

    card_delta = c.ACTIVE_CARD_MAX * 0.5
    card_start = c.ACTIVE_CARD_MAX - card_delta
    card_end = c.ACTIVE_CARD_MAX + card_delta

    df_up = df_up_full[(df_up_full.ACTIVE_CARD_MAX>=card_start)&
                   (df_up_full.ACTIVE_CARD_MAX<=card_end)&
                    (df_up_full.DRAW_UP_DATE >= pd.to_datetime(date_start))&
                  (df_up_full.DRAW_UP_DATE <= pd.to_datetime(date_end))].copy()
    
    df_up['distance'] = 0.0
    df_up.distance = df_up.apply(lambda x: Levenshtein.ratio(x['CUSTOMER'],c.CUSTOMER),axis=1)
    df_up.dropna(subset=['distance'], inplace=True)
    
    df_up = df_up[df_up.distance>0.8]
    
    for index_up, row_up in df_up.iterrows():

        customer = row_up['CUSTOMER']
        draw_up_date = row_up['DRAW_UP_DATE']
        active_card_max = row_up['ACTIVE_CARD_MAX']

        c.Match_Draw_Up_Customer(customer, draw_up_date, active_card_max)

    if len(c.MATCHING_CUSTOMERS)==1:

        if not c.CUSTOMER in (_processed_customers):

            to_save_counter += 1

            _direct_customer.append(c.CUSTOMER)
            _processed_customers.append(c.CUSTOMER)
            _direct_match.append(c.MATCHING_CUSTOMERS[0])
            _processed_customers.append(c.MATCHING_CUSTOMERS[0])
            _direct_draw_up_date.append(c.DRAW_UP_DATE[0])

            if verbose:
                print()
                print("DIRECT")
                print(c.CUSTOMER, c.WORD_LIST)
                print(c.MATCHING_CUSTOMERS)
                print(c.PERCENT_DIFFERENCE)
                print(c.DAYS_DIFFERENCE)
                print("=====")
                print()

    elif len(c.MATCHING_CUSTOMERS)>1:

        if not c.CUSTOMER in (_processed_customers):

            to_save_counter += 1

            _multiple_customer.append(c.CUSTOMER)
            _processed_customers.append(c.CUSTOMER)
            _multiple_matches.append(c.MATCHING_CUSTOMERS)
            _multiple_drop_dates.append(c.DRAW_UP_DATE)

        if verbose:
            print()
            print("MULTIPLE")
            print(c.CUSTOMER, c.WORD_LIST)
            print(c.MATCHING_CUSTOMERS)
            print(c.PERCENT_DIFFERENCE)
            print(c.DAYS_DIFFERENCE)
            print("=====")
            print()

    else:

        # could not find a match, remove it from future processing
        _no_match_customer.append(c.CUSTOMER)
        _processed_customers.append(c.CUSTOMER)

    if to_save_counter>=save_every_n:

        _matching_process_log_time.append(str(pd.Timestamp.now()))
        _matching_process_log_event.append("writing datasets to snowflake")
        do_save_log(_matching_process_log_time, _matching_process_log_event)

        do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date)
        do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates)

        _matching_process_log_time.append(str(pd.Timestamp.now()))
        _matching_process_log_event.append("saved " + str(to_save_counter) + " records to snowflake.")
        do_save_log(_matching_process_log_time, _matching_process_log_event)

        to_save_counter = 0

    t1 = time.time()

    avg_duration = (((t1-t0)/idx)/60.0)

    if idx % print_every_n == 0:
        idx_remaining = len(_customers)-idx
        print("processing", idx, "current record:", c.CUSTOMER, ",", idx_remaining, "remaining")
        print(round(avg_duration,2), "avg mins per iteration",  round((avg_duration*idx_remaining)/60,2), "estimated hrs remaining")
        print(len(_direct_customer), "direct match records", len(_multiple_customer), "multiple match records", len(_no_match_customer), "no match records")
        print()

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append("writing datasets to snowflake")
do_save_log(_matching_process_log_time, _matching_process_log_event)

do_save_direct_matches(_direct_customer, _direct_match, _direct_draw_up_date)
do_save_multiple_matches(_multiple_customer, _multiple_matches, _multiple_drop_dates)

_matching_process_log_time.append(str(pd.Timestamp.now()))
_matching_process_log_event.append("saved " + str(to_save_counter) + " records to snowflake.")
do_save_log(_matching_process_log_time, _matching_process_log_event)

processing 100 current record: 14 HRS PROD INC  IN THE DARK , 33927 remaining
0.18 avg mins per iteration 104.48 estimated hrs remaining
324 direct match records 5 multiple match records 6775 no match records

processing 200 current record: 18206-HORIZON FOOD EQUIP , 33827 remaining
0.09 avg mins per iteration 52.41 estimated hrs remaining
324 direct match records 5 multiple match records 6873 no match records

processing 300 current record: 2046 POOLCORP 11 , 33727 remaining
0.06 avg mins per iteration 35.11 estimated hrs remaining
324 direct match records 5 multiple match records 6973 no match records

processing 400 current record: 3 STONE REALTY , 33627 remaining
0.05 avg mins per iteration 26.49 estimated hrs remaining
324 direct match records 5 multiple match records 7071 no match records

processing 500 current record: 3711196 CANADA INC. , 33527 remaining
0.04 avg mins per iteration 21.27 estimated hrs remaining
324 direct match records 5 multiple match records 7171 no match re

processing 4200 current record: BARCOL CONTROLS LTD , 29827 remaining
0.01 avg mins per iteration 3.08 estimated hrs remaining
324 direct match records 5 multiple match records 10716 no match records

processing 4300 current record: BATES TRANSPORTATION SERVICE INC , 29727 remaining
0.01 avg mins per iteration 3.02 estimated hrs remaining
324 direct match records 5 multiple match records 10809 no match records

processing 4400 current record: BD PLUMBING  HEATING , 29627 remaining
0.01 avg mins per iteration 2.96 estimated hrs remaining
324 direct match records 5 multiple match records 10907 no match records

processing 4500 current record: BEKA LUBE PRODUCTS INC , 29527 remaining
0.01 avg mins per iteration 2.9 estimated hrs remaining
324 direct match records 5 multiple match records 10999 no match records

processing 4600 current record: BENTON COUNTY STONE CO INC , 29427 remaining
0.01 avg mins per iteration 2.85 estimated hrs remaining
324 direct match records 5 multiple match reco

processing 8100 current record: COMPLETE LAWN CARE , 25927 remaining
0.0 avg mins per iteration 1.78 estimated hrs remaining
374 direct match records 5 multiple match records 14400 no match records

processing 8200 current record: CONLEY SALES COMPANY , 25827 remaining
0.0 avg mins per iteration 1.76 estimated hrs remaining
381 direct match records 5 multiple match records 14493 no match records

processing 8300 current record: CONSUMERS PETROLEUM OF CONNECTICUT INCORPORATED , 25727 remaining
0.0 avg mins per iteration 1.75 estimated hrs remaining
386 direct match records 7 multiple match records 14586 no match records

processing 8400 current record: COPELAND GROUP , 25627 remaining
0.0 avg mins per iteration 1.73 estimated hrs remaining
388 direct match records 7 multiple match records 14684 no match records

processing 8500 current record: CORRIDOR NETWORK CONSTRUCTION , 25527 remaining
0.0 avg mins per iteration 1.71 estimated hrs remaining
389 direct match records 7 multiple match

processing 11700 current record: EXTERIOR TECHNOLOGIES ROOFING AND COATING LLC , 22327 remaining
0.0 avg mins per iteration 1.31 estimated hrs remaining
579 direct match records 8 multiple match records 17792 no match records

processing 11800 current record: F5 SECURITY LLC , 22227 remaining
0.0 avg mins per iteration 1.3 estimated hrs remaining
583 direct match records 8 multiple match records 17888 no match records

processing 11900 current record: FAR WEST MARINE ELECTRIC , 22127 remaining
0.0 avg mins per iteration 1.29 estimated hrs remaining
586 direct match records 8 multiple match records 17985 no match records

24 rows successfully written (UH9CBOLQy0)


saving 592 1-1 matching records

592 rows successfully written (aoZd0yo3YC)


saving 8 1-n matching records

8 rows successfully written (NyjW0BkbGX)

25 rows successfully written (6uvbQne03g)

processing 12000 current record: FERRARO FINE FOODS CORP , 22027 remaining
0.0 avg mins per iteration 1.28 estimated hrs remaining
59

processing 15200 current record: HOME ENGERGY OF LONG ISLAND LL , 18827 remaining
0.0 avg mins per iteration 0.99 estimated hrs remaining
763 direct match records 8 multiple match records 21107 no match records

processing 15300 current record: HORIZON FOOD SERVICE  SUPPLY LTD , 18727 remaining
0.0 avg mins per iteration 0.99 estimated hrs remaining
765 direct match records 8 multiple match records 21205 no match records

processing 15400 current record: HPT TRS WYN INC , 18627 remaining
0.0 avg mins per iteration 0.98 estimated hrs remaining
769 direct match records 8 multiple match records 21301 no match records

processing 15500 current record: HUSKY MAINTENANCE LLC , 18527 remaining
0.0 avg mins per iteration 0.97 estimated hrs remaining
776 direct match records 8 multiple match records 21394 no match records

processing 15600 current record: I G I SECURITY LOCKSMITH SVCS INC , 18427 remaining
0.0 avg mins per iteration 0.96 estimated hrs remaining
780 direct match records 8 multip

941 rows successfully written (mK2L38WZCk)


saving 9 1-n matching records

9 rows successfully written (d5HLa5LPpv)

39 rows successfully written (hVfQ4kx6Zq)

processing 18800 current record: LETS GET MOVING LLLP , 15227 remaining
0.0 avg mins per iteration 0.75 estimated hrs remaining
944 direct match records 9 multiple match records 24525 no match records

processing 18900 current record: LIFE SAFETY SYSTEMS , 15127 remaining
0.0 avg mins per iteration 0.74 estimated hrs remaining
949 direct match records 9 multiple match records 24620 no match records

processing 19000 current record: LINER GUYS LLC , 15027 remaining
0.0 avg mins per iteration 0.74 estimated hrs remaining
954 direct match records 9 multiple match records 24715 no match records

processing 19100 current record: LOCAL SEED COMPANY LLC , 14927 remaining
0.0 avg mins per iteration 0.73 estimated hrs remaining
958 direct match records 9 multiple match records 24810 no match records

processing 19200 current record: LOR

46 rows successfully written (pES8jMEPv7)


saving 1141 1-1 matching records

1141 rows successfully written (z9Q7YtXsyU)


saving 9 1-n matching records

9 rows successfully written (UsjRyottyE)

47 rows successfully written (1lQYfbQh4E)

processing 22400 current record: NORTHWEST CONCRETE CUTTING CORP , 11627 remaining
0.0 avg mins per iteration 0.54 estimated hrs remaining
1146 direct match records 9 multiple match records 27922 no match records

processing 22500 current record: NUMET MACHINING TECHNIQUES INC , 11527 remaining
0.0 avg mins per iteration 0.54 estimated hrs remaining
1150 direct match records 9 multiple match records 28018 no match records

processing 22600 current record: OAKLEY COMMERCIAL FLOORING CONTRACTORS , 11427 remaining
0.0 avg mins per iteration 0.53 estimated hrs remaining
1157 direct match records 9 multiple match records 28111 no match records

processing 22700 current record: OHOPAKI GENERAL CONTRACTING , 11327 remaining
0.0 avg mins per iteration 0.53 e

processing 25900 current record: RELIANCE FIRE PROTECTION 3 , 8127 remaining
0.0 avg mins per iteration 0.36 estimated hrs remaining
1315 direct match records 10 multiple match records 31251 no match records

processing 26000 current record: RESTAGE INC , 8027 remaining
0.0 avg mins per iteration 0.36 estimated hrs remaining
1319 direct match records 10 multiple match records 31347 no match records

processing 26100 current record: RICHARD L JONES CALEXIO , 7927 remaining
0.0 avg mins per iteration 0.35 estimated hrs remaining
1323 direct match records 10 multiple match records 31443 no match records

processing 26200 current record: RIO DEL FIRE PROTECTION , 7827 remaining
0.0 avg mins per iteration 0.35 estimated hrs remaining
1325 direct match records 10 multiple match records 31541 no match records

processing 26300 current record: ROAD LEGENDS COURIER INC , 7727 remaining
0.0 avg mins per iteration 0.34 estimated hrs remaining
1332 direct match records 10 multiple match records 31

processing 29500 current record: SUPERHERO FIRE PROTECTION LLC , 4527 remaining
0.0 avg mins per iteration 0.2 estimated hrs remaining
1476 direct match records 10 multiple match records 34690 no match records

processing 29600 current record: SUPREME SYSTEMS INC , 4427 remaining
0.0 avg mins per iteration 0.19 estimated hrs remaining
1480 direct match records 10 multiple match records 34786 no match records

processing 29700 current record: SYMMETRIC LLC , 4327 remaining
0.0 avg mins per iteration 0.19 estimated hrs remaining
1489 direct match records 10 multiple match records 34877 no match records

60 rows successfully written (iAtuMuvc3B)


saving 1490 1-1 matching records

1490 rows successfully written (Tdzm3G1V9M)


saving 10 1-n matching records

10 rows successfully written (9lwe8qmTWh)

61 rows successfully written (pQ6mxc7CSe)

